In [1]:
import pandas as pd
import numpy as np
import glob
import os
import random
import pickle
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import class_weight
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import datetime
import tensorflow.keras as keras
from tensorflow.keras import Sequential
from tensorflow.keras import regularizers
from tensorflow.keras.layers import LSTM, Dense, LSTM, Flatten, BatchNormalization, Dropout
from keras.utils import to_categorical
import tensorflow as tf
# Load the TensorBoard notebook extension
%load_ext tensorboard

Using TensorFlow backend.


In [2]:
countriesOfInterest = ["HK", "JP", 'ZA', 'TN', 'TR', 'GB', 'MX', 'US', 'CO', 'EC', 'AU', 'NZ']
countriesOfInterest = ["ZA", "EG", "TW", "JP", "DK", "FI", "US", "CA", "AU", "NZ", "BR", "CO"]
train_n = 500
val_n = 20
Category = "Country"
w_length = 300
enc = OneHotEncoder()
enc.fit(np.array(countriesOfInterest).reshape(-1, 1))

OneHotEncoder(categories='auto', drop=None, dtype=<class 'numpy.float64'>,
              handle_unknown='error', sparse=True)

In [28]:
def getSample(t, n):
    samples = []
    labels = []
    for country in enc.categories_[0]:
        allTracks = pickle.load(open( "Samples\\" + country + "_" + t + ".p", "rb" ))
        tracks = np.random.choice(allTracks.track_id.unique(), size=n, replace=True)
        for track in tracks:
            Feats = allTracks.loc[allTracks.track_id == track]
            ind = random.randrange(1, Feats.shape[0] - 300)
            samples = samples + [np.array(Feats[ind:ind+300].iloc[:,0:24])]
            labels = labels + [country]
    return np.array(samples), np.array(labels)

In [29]:
train_x, train_labels = getSample("train", 1)
train_x.shape

KeyboardInterrupt: 

### Fit model

In [27]:
enc.categories_[0]

array(['AU', 'BR', 'CA', 'CO', 'DK', 'EG', 'FI', 'JP', 'NZ', 'TW', 'US',
       'ZA'], dtype='<U2')

In [ ]:

model = keras.Sequential()
model.add(LSTM(64, 
               input_shape=(train_x.shape[1], train_x.shape[2]), 
               return_sequences = True
              ))
model.add(BatchNormalization())
model.add(LSTM(128, 
               input_shape=(train_x.shape[1], train_x.shape[2]), 
               return_sequences = True,
#               dropout = .5,
              go_backwards = True))
model.add(BatchNormalization())
model.add(LSTM(256, 
               input_shape=(train_x.shape[1], train_x.shape[2]),
#               dropout = .5,
 #              activity_regularizer = regularizers.l2(0.01)
              ))
model.add(BatchNormalization())
#model.add(Dropout(.5))
#model.add(Dense(64, activation = "relu"))
#model.add(BatchNormalization())
#model.add(Dropout(.5))
model.add(Dense(len(enc.categories_[0]), activation= "softmax"))
adam = keras.optimizers.Adam(lr=0.001)
model.compile(loss = "categorical_crossentropy", optimizer= adam, metrics=["acc"])
print(model.summary())

In [ ]:
desc = "64_128_256_liveSample"
log_dir = os.path.join(
    "logs",
    "fit",
    desc
)
model_dir = os.path.join(
    "pickle",
    "save"
)

train_n = 10000
val_n = 2000
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
epochs = 25
iterations = 10
learn_rate = 0.001
for i in range(iterations):
    adam = keras.optimizers.Adam(lr=learn_rate)
    model.compile(loss = "categorical_crossentropy", optimizer= adam, metrics=["acc"])
    train_x, train_labels, val_x, val_labels, class_weights = getSamples(train_n, val_n)
    print(np.sum(train_labels, axis = 0))
    model.fit(train_x, train_labels,
              epochs = i * epochs + epochs, 
              initial_epoch = i * epochs,
              shuffle = True,
              validation_data = (val_x, val_labels),
              batch_size = 1024,
              class_weight = class_weights,
             callbacks=[tensorboard_callback],
             verbose = 1)
    model.save_weights(model_dir)
    if i%2 == 0:
        learn_rate = learn_rate/2
    if i % 1 == 0:
        preds = model.predict(val_x, batch_size = 1024, verbose = 1)
     #   print(np.sum(train_labels, axis = 0))
        plt.imshow(
            confusion_matrix(
                enc.inverse_transform(preds), 
                enc.inverse_transform(val_labels), 
               # normalize = "all"
            )
        )
        plt.pause(.5)
        plt.show()
        preds = model.predict(train_x, batch_size = 1024, verbose = 1)
        plt.imshow(
            confusion_matrix(
                enc.inverse_transform(preds), 
                enc.inverse_transform(train_labels), 
            #    normalize = "all"
            )
        )
        plt.pause(.5)
        plt.show()